<a href="https://colab.research.google.com/github/Ilank1/BillsPrediction/blob/main/DataPreparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wget
!pip install python-docx

import pandas as pd
import re, os, wget
from docx import Document

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import re

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

def download_from_googledrive(url):
  regex = "([\w-]){33}|([\w-]){19}"
  file_downloaded = drive.CreateFile({'id': re.search(regex,url).group(0)})
  filename = file_downloaded['title']
  print("Downloading: ", filename)
  file_downloaded.GetContentFile(filename)
  return filename

kns_bill_url = "https://drive.google.com/file/d/1rt330HT93JR1ypRRgNZYDYxjx3wrjcvJ/view?usp=sharing"
db_Bill_path = download_from_googledrive(kns_bill_url)

documents_bill_url = "https://drive.google.com/file/d/1rg6OMELqXI3KWr6WNWN4Qlubgh0f93y-/view?usp=sharing"
documents_db_path = download_from_googledrive(documents_bill_url)


Downloading:  KNS_Bill.csv
Downloading:  KNS_DocumentBill.csv


In [28]:
passed_status = 118

SubTypeOptions = {"ועדה":0, 
       "פרטית":1,
       "ממשלתית":2}

#documents_db_path = r"KNS_DocumentBill.csv"
documents_dir = r"/content/Bills"
#db_Bill_path = r"KNS_Bill.csv"

Download all the documents of the bills from the govermental OData API
http://knesset.gov.il/Odata/ParliamentInfo.svc/

We've dumped the following tables for preparing the data 
KNS_DocumentBill - For the documents
KNS_Bill - For bills info

In [4]:
df_documents = pd.read_csv(documents_db_path, encoding='utf-8-sig')

In [29]:
print(os.listdir("."))
#os.chdir("/")
import shutil
#shutil.rmtree("/Bills")

import glob

files = glob.glob('*.docx', recursive=True)

for f in files:
    try:
        os.remove(f)
    except OSError as e:
        print("Error: %s : %s" % (f, e.strerror))

[]


In [38]:
if not os.path.exists(documents_dir):
  os.makedirs(documents_dir)
os.chdir(documents_dir)
for index, row in df_documents.iterrows():
    # Download only DOC files (doc or docx) and initial suggestion type of documents(type 1)
    if row["ApplicationDesc"] == "DOC" and row["GroupTypeID"] == 1:
        file_name = wget.filename_from_url(row["FilePath"])

        file_path = os.path.join(documents_dir, file_name)
        
        # Download non-existing files
        if os.path.isfile(file_path):
            continue
        else:
            file_name = wget.filename_from_url(row["FilePath"])
            if ".docx" not in file_name:
              continue
            file_name = wget.download(row["FilePath"])
            
            print(" : " + file_name)

print("Finished download")

 : 19_lst_239562.docx
 : 19_lst_239563.docx
 : 19_lst_239564.docx
 : 19_lst_239565.docx
 : 19_lst_239566.docx
 : 19_lst_239567.docx
 : 19_lst_239568.docx
 : 19_lst_239569.docx
 : 19_lst_239570.docx
 : 19_lst_239571.docx
 : 19_lst_239572.docx
 : 19_lst_239573.docx
 : 19_lst_240353.docx


KeyboardInterrupt: ignored

In [35]:
def get_bill_text(bill_id):
    bill_df = df_documents.loc[df_documents['BillID'] == bill_id]
    if len(bill_df) == 0:
        raise Exception("No documents found for bill id {}".format(bill_id))
    for i, r in bill_df.iterrows():
        
        # Use only DOC documents, those are the only type of documents we've downloaded
        if r["ApplicationDesc"] != "DOC":
            raise Exception("Bill document type {} not supported, only DOC supported".format(r["ApplicationDesc"]))
        
        # Use only type 1 documents, those are the only type of documents we've downloaded
        if r["GroupTypeID"] != 1:
            raise Exception("Bill GroupTypeID {} not supported, only initial suggesstions are allowed".format(r["GroupTypeID"]))
        
        file_path = os.path.join(documents_dir, wget.filename_from_url(r["FilePath"]))
        if not os.path.isfile(file_path):
            raise Exception("Bill document {} doesn't exists in the directory, web path {}".format(file_path, r["FilePath"]))
        
        return extract_bill_explanation(file_path)
    

In [34]:
def extract_bill_explanation(file_path):
    text = ""
    document = Document(file_path)

    start_ind = [i for i, para in enumerate(document.paragraphs) if (('דברי הסבר' in para.text) or ('דברי  הסבר' in para.text) or ('ד ב ר י   ה ס ב ר' in para.text))]
    if len(start_ind) == 0:
            raise Exception("No Explanataion paragraph headline found")
    if len(start_ind) > 1:
        raise Exception("More than one Explanataion paragraph headline found")
    else:
        start_ind = start_ind[0]

    end_ind = [i for i, para in enumerate(document.paragraphs) if '---' in para.text]
    if len(end_ind) == 0:
        raise Exception("No Explanataion paragraph headline found")
    if len(end_ind) > 1:
        raise Exception("More than one Explanataion paragraph end marker found")
    else:
        end_ind = end_ind[0]
    
    if start_ind and end_ind:
        for i, para in enumerate(document.paragraphs):
            if i > start_ind  and i < end_ind :
                 text += para.text + "\r\n"
    return text

In [42]:
df_bills = pd.read_csv(os.path.join(r"/content", db_Bill_path), encoding='utf-8-sig')

outout_df = pd.DataFrame(columns=['label', 'data','InitiatorType','cont_bit', 'bill_id'])

for index, row in df_bills.iterrows():
    try:
                                       
        current_bill = {'label': row["StatusID"] == passed_status, 
                        'data': get_bill_text(row["BillID"]),
                        'InitiatorType': SubTypeOptions[row["SubTypeDesc"]],
                        'cont_bit': row["IsContinuationBill"],
                        'bill_id': row["BillID"]}
        
        outout_df = outout_df.append(current_bill,ignore_index=True)
    except Exception as e: 
        print(e)
        print("Failed bill id: ", row["BillID"])

outout_df.replace({False: 0, True: 1}, inplace=True)

outout_df.to_excel(r"/content/output.xlsx") 

print("Finish")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Streaming output truncated to the last 5000 lines.
Failed bill id:  2085778
Bill document /content/Bills/23_lst_570290.docx doesn't exists in the directory, web path https://fs.knesset.gov.il/23/law/23_lst_570290.docx
Failed bill id:  2085779
Bill document /content/Bills/23_lst_573578.docx doesn't exists in the directory, web path https://fs.knesset.gov.il/23/law/23_lst_573578.docx
Failed bill id:  2085780
Bill document /content/Bills/23_lst_568621.docx doesn't exists in the directory, web path https://fs.knesset.gov.il/23/law/23_lst_568621.docx
Failed bill id:  2085782
Bill document /content/Bills/23_lst_568914.docx doesn't exists in the directory, web path https://fs.knesset.gov.il/23/law/23_lst_568914.docx
Failed bill id:  2085785
Bill document /content/Bills/23_lst_568639.docx doesn't exists in the directory, web path https://fs.knesset.gov.il/23/law/23_lst_568639.docx
Failed bill id:  2085787
Bill document /content/Bills/23_lst_568640.docx doesn't exists in the directory, web path

Final stats regarding the data: 

In [47]:
all_bills = r"/content/output.xlsx"
df_all_bills = pd.read_excel(all_bills)

total = len(df_all_bills)
print("Total: ", total)

print("All extracted bills: ", len(df_all_bills))

df_bills_cont = df_all_bills.loc[df_all_bills["cont_bit"] == 1]
print("Continuation bit on ", len(df_bills_cont))

df_bills_notcont = df_all_bills.loc[df_all_bills["cont_bit"] == 0]
print("Continuation bit off ", len(df_bills_notcont))



df_bills_com = df_all_bills.loc[df_all_bills["InitiatorType"] == 0]
print("Committee bills: ", len(df_bills_com))

df_bills_gov = df_all_bills.loc[df_all_bills["InitiatorType"] == 2]
print("Govermental bills: ", len(df_bills_gov))

df_bills_private = df_all_bills.loc[df_all_bills["InitiatorType"] == 1]
print("Private bills: ", len(df_bills_private))


df_all_bills = df_all_bills.loc[df_all_bills["label"] == 1]
print("Passed: " ,len(df_bills))

df_bills_cont = df_all_bills.loc[df_all_bills["cont_bit"] == 1]
print("Continuation bit on ", len(df_bills_cont))

df_bills_notcont = df_all_bills.loc[df_all_bills["cont_bit"] == 0]
print("Continuation bit off ", len(df_bills_notcont))

df_bills_com = df_all_bills.loc[df_all_bills["InitiatorType"] == 0]
print("Committee bills: ", len(df_bills_com))

df_bills_gov = df_all_bills.loc[df_all_bills["InitiatorType"] == 2]
print("Govermental bills: ", len(df_bills_gov))

df_bills_private = df_all_bills.loc[df_all_bills["InitiatorType"] == 1]
print("Private bills: ", len(df_bills_private))

Total:  214
All extracted bills:  214
Continuation bit on  0
Continuation bit off  0
Committee bills:  0
Govermental bills:  0
Private bills:  214
Passed:  47246
Continuation bit on  0
Continuation bit off  0
Committee bills:  0
Govermental bills:  0
Private bills:  20
